# Keras Tuner 소개
- https://www.tensorflow.org/tutorials/keras/keras_tuner
- keras_tuner: TensorFlow 프로그램을 위한 최적의 하이퍼파라미터 세트를 선택하는 데 도움이 되는 라이브러리

하이퍼파라미터: 학습 프로세스와 ML 모델의 토폴로지를 제어하는 변수
1. 모델 하이퍼파라미터: 히든 레이어 수, 너비 등
2. 알고리즘 하이퍼파라미터: SGD(Stochastic Gradient Descnet) learning ratio, KNN의 이웃수

In [1]:
import tensorflow as tf 
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner

In [3]:
import kerastuner as kt 

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

hypermodel 정의 (hypertuning 할 모델): 모델 아키텍쳐 + 하이퍼파라미터 search space
- model builder function 이용
- KerasTuner API에서 HyperModel의 서브 클래스 생성

pre-defined HyperModel도 이용 가능: HyperXception, HyperResNet for computer vision applications.

model builder function
- compiled model을 리턴
- uses hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    # model hyperparameter
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32) # 32~512에서 32 단위로 최적 값 탐색
    model.add(keras.layers.Dense(units=hp_units, activation='relu')) # 이 레이어에서 최적 값 찾을 것
    model.add(keras.layers.Dense(10)) # 10 classes
    # set model architecture + hyperparameter search space 

    # algorithm hyperparameter
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) # 3개 중 최적 값 탐색 
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

keras Tuner가 사용 가능한 tuner들: RandomSearch, Hyperband, BayesianOptimization, Sklearn -> hyperband를 예시로
- hyperband tuner를 instantiate하려면 hypermodel(최적화할 objective와 max_epochs)를 정해야 함

In [9]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3, 
                     directory='my_dir', project_name='intro_to_kt')
# tuner: hyperparameter를 어떻게 찾아갈 것인가? 

adaptive resource allocation & early stopping -> 빠르게 수렴 (성능 좋아야 다음 라운드로 전달 )

In [10]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [11]:
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test), 
             callbacks=[ClearTrainingOutput()])

Trial 30 Complete [00h 00m 17s]
val_accuracy: 0.8835999965667725

Best val_accuracy So Far: 0.8835999965667725
Total elapsed time: 00h 05m 22s
INFO:tensorflow:Oracle triggered exit


In [12]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [15]:
model = tuner.hypermodel.build(best_hps)

In [16]:
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 884us/step - loss: 0.6304 - accuracy: 0.7803 - val_loss: 0.4313 - val_accuracy: 0.8471
Epoch 2/10
1875/1875 [==============================] - 2s 841us/step - loss: 0.3812 - accuracy: 0.8625 - val_loss: 0.3889 - val_accuracy: 0.8595
Epoch 3/10
1875/1875 [==============================] - 2s 826us/step - loss: 0.3353 - accuracy: 0.8768 - val_loss: 0.3876 - val_accuracy: 0.8590
Epoch 4/10
1875/1875 [==============================] - 2s 857us/step - loss: 0.3074 - accuracy: 0.8888 - val_loss: 0.3655 - val_accuracy: 0.8679
Epoch 5/10
1875/1875 [==============================] - 2s 829us/step - loss: 0.2944 - accuracy: 0.8904 - val_loss: 0.3905 - val_accuracy: 0.8610
Epoch 6/10
1875/1875 [==============================] - 2s 831us/step - loss: 0.2788 - accuracy: 0.8962 - val_loss: 0.3432 - val_accuracy: 0.8780
Epoch 7/10
1875/1875 [==============================] - 2s 835us/step - loss: 0.2653 - accuracy: 0.9014 - val_loss: 0.3438 -

- my_dir/intro_to_kt 디렉토리에는 하이퍼 파라미터 검색 중에 실행 된 모든 시험 (모델 구성)에 대한 자세한 로그와 체크 포인트가 포함되어 있습니다. 
- 하이퍼 파라미터 검색을 다시 실행하면 Keras Tuner는 이러한 로그의 기존 상태를 사용하여 검색을 재개합니다. 
- 이 동작을 비활성화하려면 튜너를 인스턴스화하는 동안 추가 overwrite = True 인수를 전달하십시오.